In [1]:
!pip install transformers datasets pandas peft -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
from datasets import Dataset

# Read the CSV file (replace "dataset.csv" with your file name)
df = pd.read_csv("dataset.csv")

# Combine riddle and solution into a single text column
df["text"] = "Riddle: " + df["Riddle"] + "\nAnswer: " + df["Solution"].astype(str) + "\n"

# Convert to a Hugging Face Dataset and split into train/test
dataset = Dataset.from_pandas(df[["text"]])
dataset = dataset.train_test_split(test_size=0.1)

In [4]:
from transformers import GPT2Tokenizer

# Load GPT-2 tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding

# Define tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=128, padding="max_length")

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [5]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, TaskType

# Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained(model_name)

# Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # For causal language modeling (like GPT-2)
    r=8,  # Rank of the low-rank matrices (controls adapter size)
    lora_alpha=32,  # Scaling factor for LoRA
    lora_dropout=0.1,  # Dropout rate for LoRA layers
    target_modules=["c_attn"],  # Apply LoRA to attention layers in GPT-2
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Set up data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-lora-finetuned-riddles",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="no",  # Skip evaluation for simplicity
    save_steps=500,
    save_total_limit=2,
    logging_steps=50,
    learning_rate=5e-4,  # Higher learning rate suitable for PEFT
    report_to=[],  # Disable external logging (e.g., wandb)
)

# Initialize and run the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
)

trainer.train()

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=42, training_loss=2.899307432628813, metrics={'train_runtime': 3.5397, 'train_samples_per_second': 22.883, 'train_steps_per_second': 11.865, 'total_flos': 5309509533696.0, 'train_loss': 2.899307432628813, 'epoch': 3.0})

In [6]:
model.save_pretrained("./gpt2-lora-finetuned-riddles")
tokenizer.save_pretrained("./gpt2-lora-finetuned-riddles")

('./gpt2-lora-finetuned-riddles/tokenizer_config.json',
 './gpt2-lora-finetuned-riddles/special_tokens_map.json',
 './gpt2-lora-finetuned-riddles/vocab.json',
 './gpt2-lora-finetuned-riddles/merges.txt',
 './gpt2-lora-finetuned-riddles/added_tokens.json')

In [7]:
from peft import PeftModel

# Load the base model and apply the LoRA adapters
model = GPT2LMHeadModel.from_pretrained(model_name)
model = PeftModel.from_pretrained(model, "./gpt2-lora-finetuned-riddles")

# Prepare the prompt
prompt = "Riddle: "
encoding = tokenizer(prompt, return_tensors="pt")
input_ids = encoding.input_ids
attention_mask = encoding.attention_mask

# Generate 5 new riddles
generated_outputs = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_length=100,
    num_return_sequences=5,
    do_sample=True,  # Enable sampling for creativity
    temperature=0.7,  # Control randomness
    top_p=0.9,  # Use nucleus sampling
    pad_token_id=tokenizer.eos_token_id,
)

# Decode and print the generated riddles
for i, output in enumerate(generated_outputs):
    text = tokenizer.decode(output, skip_special_tokens=True)
    print(f"Generated Riddle {i+1}:\n{text}\n")

Generated Riddle 1:
Riddle:  When the number of people is 12, does that mean 2,000?
Answer:  2
Answer: 4
Answer: 7
Answer: 8
Answer: 9
Answer: 10
Answer: 11
Answer: 12
Answer: 13
Answer: 14
Answer: 15
Answer: 16
Answer: 17
Answer: 18
Answer: 19
Answer: 20
Answer: 21
Answer: 22
Answer: 23
Answer

Generated Riddle 2:
Riddle:  When will it be 6pm?
Answer:  7
Answer:  8
Answer:  9
Answer:  10
Answer:  11
Answer:  12
Answer:  13
Answer:  14
Answer:  15
Answer:  16
Answer:  17
Answer:  18
Answer:  19
Answer:  20
Answer:

Generated Riddle 3:
Riddle:  How many times does a person have to wait for the person to pass out?
Answer: 6
Answer: 10
Answer: 10
Answer: 12
Answer: 13
Answer: 12
Answer: 12
Answer: 12
Answer: 12
Answer: 12
Answer: 12
Answer: 12
Answer: 12
Answer: 12
Answer: 12
Answer: 12
Answer: 12
Answer: 12
Answer: 12
Answer:

Generated Riddle 4:
Riddle:  When do I die?
Answer:  8
Answer:  4
Answer:  10
Answer:  11
Answer:  12
Answer:  13
Answer:  14
Answer:  15
Answer:  16
Answer:  17
A

In [8]:
from peft import PeftModel
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
model = PeftModel.from_pretrained(model, "./gpt2-lora-finetuned-riddles")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-lora-finetuned-riddles")

def generate_solution(riddle):
    """
    Generate a solution for the given riddle using the fine-tuned model.

    Args:
        riddle (str): The input riddle provided by the user.

    Returns:
        str: The generated solution extracted from the model's output.
    """
    # Format the input in the same style as the training data
    input_text = f"Riddle: {riddle}\nAnswer:"

    # Tokenize the input for the model
    encoding = tokenizer(input_text, return_tensors="pt")
    input_ids = encoding.input_ids
    attention_mask = encoding.attention_mask

    # Generate the solution using the model
    generated_output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=50,  # Limit the solution to 50 new tokens
        do_sample=True,    # Use sampling for creative outputs
        temperature=0.7,   # Balance randomness and coherence
        top_p=0.9,         # Use nucleus sampling for better diversity
        pad_token_id=tokenizer.eos_token_id,  # Ensure proper padding
    )

    # Decode the generated output into readable text
    decoded_output = tokenizer.decode(generated_output[0], skip_special_tokens=True)

    # Extract the solution (text after "Answer:")
    parts = decoded_output.split("\nAnswer:")
    if len(parts) > 1:
        solution = parts[1].strip()  # Take the part after "Answer:" and remove extra spaces
    else:
        solution = "Could not generate a solution."

    return solution

# Prompt the user for a riddle and display the solution
riddle = input("Enter a riddle: ")
solution = generate_solution(riddle)
print(f"Generated Solution:\n{solution}")

Enter a riddle: c
Generated Solution:



In [10]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 22 packages in 3s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸

In [11]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.8 MB/s eta 0:00:00


In [9]:
!wget -q -O - ipv4.icanhazip.com

35.230.60.77


In [ ]:
!streamlit run app.py &>/dev/null & npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦your url is: https://odd-owls-sneeze.loca.lt
